# Neutron Star Class SPH Code

Code based on the tutorials here: 

https://philip-mocz.medium.com/create-your-own-smoothed-particle-hydrodynamics-simulation-with-python-76e1cec505f1

https://github.com/zaman13/Modeling-of-Neutron-Stars/

### Notes: 
- Calculates density from mass times the gradient of the smoothing function.
- Defines 3D Gausssian Smoothing kernel 
- Euler's Method for iterating pressure, density as a function of distance


In [141]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
import time
import math
import matplotlib as mp
import scipy as sp
import pylab as py

from ipynb.fs.full.Neutron_Star_SPH_sph import W, gradW, getPairwiseSeparations, getDensity
from ipynb.fs.full.Neutron_Star_SPH_eos import EOS, initial_n, rho_NS

rho_s = 1665.3 #  Central density (density at r = 0)
hc = 197.327           # Conversion factor in MeV fm (hut * c)
G = hc * 6.67259e-45   # Gravitational constant
Ms = 1.1157467e60      # Mass of the sun in kg
mn = 938.926           # Mass of neutron in MeV c^-2

In [150]:
def EulerSolver(r,m,p,h,eos, flag = False):
    """ Euler Solver which iterates P and mass"""
    y = np.zeros(2)
    dp_dr = eos.dp_dr # extract the relevant dp_dr from the given EOS.
    dm_dr = eos.dm_dr 
    y[0] = m + dm_dr(r,m,p)*h
    y[1] = p + dp_dr(r,m,p,flag)*h
    print(dp_dr(r,m,p,flag))
    print("l", r,m,p)
    print("y1!!", y[1])
    return y

def getPressure(rho, eos, star, tol = 9e-5):
    """ 
    Takes in a star and an eos. 
    Calcualates the pressure at each point in the star. 
    Returns a vector with shape (num_points x 1).
    """
    
    ## Exctract relavent values from the star & EOS
    N = star.num_points
    m = star.m
    k = eos.constants["k"][0]
    n = eos.constants["n"][0]
    rho_s = eos.constants["rho_s"][0]
    ni = initial_n(eos)

    #### PREESURE AND DENSITY ITERATION OVER SPACE
    r1 = np.linspace(0,15,N) # why 15? Oh. 15 blocks where the density will be evaluated inside the star??
    r = r1
    h = r[1]-r[0] # we have re-defined the smoothing length here. Not sure if this will cause problems...
    m_ = np.zeros(N) # each data point has a mass. They are all 0 at the moment.
    p = np.zeros(N) # each data point has a pressure. They are all 0 at the moment.
    rh = np.zeros(N)
    ni = initial_n(eos)

    # Initial values
    r1[0] = 0
    m_[0] = 0 # 
    p[0] = 363.44 * (ni**2.54)/rho_s
    rh[0] = 1
    mf = 0
    rf = 0

    print("Initial number density, ni = ", ni)
    print("Initial Pressure, P[0] = ", p[0])
    print("Simulation range, R = 0 to ", r[-1]*eos.R0*1e-18, "km") 
    tol = 9e-5

    # Only classical model is used here
    for k in range(0,2):
        flag = 0
        for i in range(0,N-1): # for each point
            [m_[i+1], p[i+1]] = EulerSolver(r[i],m_[i],p[i],h,eos,False) # Euler's Method
            if i < 10:
                print(r[i],m_[i],p[i])
            rh[i+1] = rho_NS(r[i])
            if p[i+1] < tol:
                rf = r[i]
                mf = m_[i]
                break
            
        if i == N-2:
            print("Program didn't converge to P = 0, extend the maximum value of r")
        else:
            print("P <", tol, "found after", i, "runs")
    
        m_ = m_[0:i+2] # Update the mass of each point
        p = p[0:i+2] # Update the pressure of each point
        rh = rh[0:i+2]
        r = r[0:i+2]
        lbl = "Euler's Method with h = " + str(h)
        clr = "red"
    
    print("==============================================")
    print("Initial density, rho_s = ", rho_s, "MeV/fm3")
    print("Total mass = ", mf*eos.M0/Ms, "times Solar mass")
    print("Radius of the Neutron star = ", rf*eos.R0*1e-18, "km")

    ## p is not going to have the same shape as the star points
    ### Need to extrapolate the pressures to the points where the stars are, then return p
    return p

def getAcc(star, m,h, eos, lmbda, nu):
    """ Calculate the acceleration on each SPH particle given current positions and velocities.
    
    'pos' contains particle coordinates (N x 3 matrix)
    'vel' contains particle velocities (N x 3 matrix)
    returns 'a' containing accelerations (another N x 3 matrix)
    
    m = particle mass, h = smoothing length, k = equation of state constant, n = polytropic index, lmbda = external force constant, nu = viscosity. 
    
    """ 
    # Exctract positions and velocities of points from the star
    pos = star.pos 
    vel = star.vel
    
    N = pos.shape[0] # Number of particles
    rho = getDensity(pos, pos, m, h) # Reconstruct the densities at the position of each particle
    P = getPressure(rho, eos, star) # Get the pressures corresponding to those densities
    dx, dy, dz = getPairwiseSeparations( pos, pos ) # Get pairwise distances
    dWx, dWy, dWz = gradW( dx, dy, dz, h )# Get pairwise gradients
    
    ax = - np.sum( m * ( P/rho**2 + P.T/rho.T**2  ) * dWx, 1).reshape((N,1)) # Add x Pressure contribution to accelerations
    ay = - np.sum( m * ( P/rho**2 + P.T/rho.T**2  ) * dWy, 1).reshape((N,1)) # Add y Pressure contribution to accelerations
    az = - np.sum( m * ( P/rho**2 + P.T/rho.T**2  ) * dWz, 1).reshape((N,1)) # Add z Pressure contribution to accelerations
    a = np.hstack((ax,ay,az)) # pack together the acceleration components
    
    ##### Extra forces
    a -= lmbda * pos # Add external potential force
    a -= nu * vel # Add viscosity
    
    return a

In [151]:
class NS:
    def __init__(self, tag, mass = 1, radius = 1, num_points = 300):
        """
        NEUTRON STAR OBJECT
        
        A neutron star has the following attributes: 
            - tag, mass, radius, num_points  (eos?)
            
        A point has the following attributes: 
            - position (N x 3), vel (N x 3), acc (N x 3), density (N x 1), pressure (N x 1).        
        
        Goal: we want to give this star it's overall attributes and calculate the point attributes automatically.
        
        """
        self.tag = tag # "Name" of the star, just a string for identification
        self.radius = radius # star radius in km
        self.mass = mass # star mass in solar Masses
        self.radius_unit = 1 # if the radius of the star is given in units other than kilometers,
        # you can give it a scale factor here.    
        
        self.num_points = num_points 
        self.m = mass/num_points
        
        # for each point in the star, initialize random positions and velocities.
        np.random.seed(42) # set the random number generator seed
        self.pos = np.random.randn(self.num_points,3)   # randomly select positions and velocit-
        # -ies from initialized seed
        self.vel = np.zeros(self.pos.shape)    # randomly select positions and velocities 
        # positions and velocities are N x 3 matrices.
        self.acc = np.zeros(self.pos.shape)  # initially we haven't updated the accelerations of
        # those random points for the star.
        self.rho = None
        self.density = None
        
    def __str__(self):
        return "\nSTAR OBJECT. Name: '" + self.tag + "', Mass: " + str(self.radius) + " SMs, "+ "Radius: " + str(self.mass*self.radius_unit) + " km"
    
    def get_Acc(self, eos):
        a = getAcc(self, self.mass, h, eos, lmbda, nu) # calculate initial gravitational 
        # accelerations
        self.acc = a
        return self.acc
        # acc is also an N x 3 matrix (so new we have pos, vel, acc for x,y,z.)
    
    def initialize_mp_distributions(self): 
        """ Initializes an array of M and p based off of the mass and radius of the star, and the EOS. The star only needs to do this once. """
    
    def reset(self, num_points): 
        """ Resets the star with a new number of points. Randomized positions of the new points. """
        self.num_points = num_points
        np.random.seed(42) # set the random number generator seed
        self.pos = np.random.randn(self.num_points,3)   # randomly select positions and velocities from initialized seed
        self.vel = np.zeros(self.pos.shape)    # randomly select positions and velocities 
        # positions and velocities are N x 3 matrices.
        self.acc = None # initially we haven't updated the accelerations of those random points for the star.
        self.rho = None
        self.density = None    
        
    def simulate(self, eos, dt = 0.04, tEnd = 12):
        N = self.num_points # Number of particles
        M = self.mass # total star mass (Solar Masses)
        R = self.radius # star radius (km)
        k = eos.constants["k"][0] # equation of state constant
        n = eos.constants["n"][0] # polytropic index
        h = 0.05 # from previous results, this seems like a good smoothing length.
        nu = 1 # damping
        lmbda = eos.lmbda(M,R)
        t = 0      # current time of the simulation
        Nt = int(np.ceil(tEnd/dt)) # number of timesteps

        for i in range(Nt): # Star Simulation Time Iteration Main Loop
            self.vel += self.acc * dt/2 # (1/2) kick # adds acceleration
            self.pos += self.vel * dt # particle motion
            self.acc = getAcc(self, self.m, h, eos, lmbda, nu) # get new Nx3 matrix of accelerations based on new pos,v
            self.vel += self.acc * dt/2 # (1/2) kick # not sure why we add accelleration again.
            t += dt # update time
            if t % 100:
                print("time: " , t)
            self.rho = getDensity(self.pos, self.pos, self.m, h) # get updated density for each point based on m, h. 
        
### Simulation
my_EOS = EOS("standard") # Specify EOS
star_test = NS("star_test") # Create a star
star_test.simulate(my_EOS)

Newton-Raphson Converged after  5 iterations
Newton-Raphson Converged after  5 iterations
Initial number density, ni =  1.2918969375342138
Initial Pressure, P[0] =  0.4182722266764404
Simulation range, R = 0 to  90.36486611870906 km
-0.0
l 0.0 0.0 0.4182722266764404
y1!! 0.4182722266764404
0.0 0.0 0.4182722266764404
-0.029760395432373302
l 0.05016722408026756 0.0 0.4182722266764404
y1!! 0.41677923025006713
0.05016722408026756 0.0 0.4182722266764404
-0.07710051244271394
l 0.10033444816053512 0.00012625837986219616 0.41677923025006713
y1!! 0.41291131156565003
0.10033444816053512 0.00012625837986219616 0.41677923025006713
-0.12754281352824245
l 0.1505016722408027 0.0006302847645849113 0.41291131156565003
y1!! 0.4065128426595509
0.1505016722408027 0.0006302847645849113 0.41291131156565003
-0.1772888236377473
l 0.20066889632107024 0.0017584616349991665 0.4065128426595509
y1!! 0.397618754517189
0.20066889632107024 0.0017584616349991665 0.4065128426595509
-0.22491379401761094
l 0.250836120401

ValueError: operands could not be broadcast together with shapes (32,) (1,300) 